In [ ]:
#Mount Google Drive with this Colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Aug 10 20:51:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Zoobot and necessary modules

In [ ]:
!git clone https://github.com/mwalmsley/zoobot.git

Cloning into 'zoobot'...
remote: Enumerating objects: 8365, done.
remote: Counting objects: 100% (2353/2353), done.
remote: Compressing objects: 100% (782/782), done.
remote: Total 8365 (delta 1619), reused 2274 (delta 1561), pack-reused 6012
Receiving objects: 100% (8365/8365), 334.18 MiB | 27.25 MiB/s, done.
Resolving deltas: 100% (5209/5209), done.


In [ ]:
!pip install -e /content/zoobot[tensorflow]

Obtaining file:///content/zoobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install keras_applications

In [ ]:
import os
import logging
import json
import glob
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tqdm import tqdm
from sklearn.dummy import DummyClassifier
from sklearn import metrics
import seaborn as sns
%load_ext tensorboard

# google colab needs this hack to change the system path to include zoobot
# not needed locally
import sys
sys.path.insert(0,'/content/zoobot')
sys.path.insert(0,'/content/galaxy-datasets')

from zoobot.shared import label_metadata, schemas
#from zoobot.tensorflow.data_utils import image_datasets
from zoobot.tensorflow.estimators import define_model, custom_layers #preprocess
from zoobot.tensorflow.predictions import predict_on_dataset
from zoobot.tensorflow.training import training_config, losses, finetune
#from zoobot.tensorflow.transfer_learning import utils

from galaxy_datasets.tensorflow import get_image_dataset, add_transforms_to_dataset
from galaxy_datasets.transforms import default_transforms
from galaxy_datasets import gz_hubble

checkpoint_dir = '/content/drive/MyDrive/MPE/ZoobotEuclid/Zoobot/evo_tf_gr_16129' #weights B
#checkpoint_dir = '/content/drive/MyDrive/MPE/ZoobotEuclid/Zoobot/dr5_tf_gr_4975' #weights A
#checkpoint_dir = None #weights C

In [ ]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'

In [ ]:
device_name = tf.test.gpu_device_name()
print("Device Name:",device_name)

Device Name: /device:GPU:0


# Images and Labels

### Images

In [ ]:
#Image parameters
initial_size, resize_size, file_format = 300, 224, "jpg"

#Batch size
batch_size = 64

In [ ]:
#Euclidized HST images
!cp /content/drive/MyDrive/MPE/ZoobotEuclid/Data/Euclidized_COSMOS_complete/jpgs_euclid_complete.zip /content/
!unzip jpgs_euclid_complete.zip

#Original HST images
#!cp /content/drive/MyDrive/MPE/ZoobotEuclid/Data/Hubble_COSMOS_complete/jpgs_hubble_complete.zip /content/
#!unzip jpgs_hubble_complete.zip

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: jpgs_euclid_complete/20057247.jpg  
  inflating: jpgs_euclid_complete/20196205.jpg  
  inflating: jpgs_euclid_complete/20110801.jpg  
  inflating: jpgs_euclid_complete/20115629.jpg  
  inflating: jpgs_euclid_complete/20128307.jpg  
  inflating: jpgs_euclid_complete/20100342.jpg  
  inflating: jpgs_euclid_complete/20154824.jpg  
  inflating: jpgs_euclid_complete/20023681.jpg  
  inflating: jpgs_euclid_complete/20116543.jpg  
  inflating: jpgs_euclid_complete/20101099.jpg  
  inflating: jpgs_euclid_complete/20014114.jpg  
  inflating: jpgs_euclid_complete/20135187.jpg  
  inflating: jpgs_euclid_complete/20005213.jpg  
  inflating: jpgs_euclid_complete/20122447.jpg  
  inflating: jpgs_euclid_complete/20108397.jpg  
  inflating: jpgs_euclid_complete/20017676.jpg  
  inflating: jpgs_euclid_complete/20208664.jpg  
  inflating: jpgs_euclid_complete/20177285.jpg  
  inflating: jpgs_euclid_complete/20053271.jpg  
  

In [ ]:
def check_for_images(paths):
    """
        Check if images are present and remove non-existing images.
    """
    print("There are labels for",len(paths),"images.")
    print("Checking if the files exist...")
    ids_checked= []
    for i in range(len(paths)):
        file_exists = os.path.exists(paths[i])
        if file_exists:
            ids_checked.append(paths[i].split("/")[-1].split(".")[0])
    print("\n --> Check over. There are actually",len(ids_checked),"images.")
    return ids_checked

### Labels

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/MPE/ZoobotEuclid/Data/gz_hubble_labels_zoobot_complete.csv") #Complete dataset
paths = ["/content/jpgs_euclid_complete/" + str(name) for name in list(df["filename"])]
df["id_str"] = paths
ids_checked = check_for_images(paths)
filenames = [str(name)+".jpg" for name in ids_checked]

There are labels for 84954 images.
Checking if the files exist...

 --> Check over. There are actually 76176 images.


In [ ]:
#Split catalog
filenames_train_val, filenames_test = train_test_split(filenames, train_size=0.8, random_state=30)
catalog_train_val = df[df["filename"].isin(filenames_train_val)]
catalog_test = df[df["filename"].isin(filenames_test)]
print("Applied 80%/20% split: {} images for train and val, {} images for testing".format(len(catalog_train_val),len(catalog_test)))

Applied 80%/20% split: 60940 images for train and val, 15236 images for testing


In [ ]:
#catalog_train_val = catalog_train_val[catalog_train_val["MAG_BEST_HI"]<22.5] #Bright
catalog_train_val #Complete

,survey_id,RA,DEC,Z_BEST,MAG_BEST_HI,filename,smooth-or-featured-hubble_smooth,smooth-or-featured-hubble_features,smooth-or-featured-hubble_artifact,disk-edge-on-hubble_yes,...,clump-count-hubble_2,clump-count-hubble_3,clump-count-hubble_4,clump-count-hubble_5-plus,clump-count-hubble_cant-tell,galaxy-symmetrical-hubble_yes,galaxy-symmetrical-hubble_no,clumps-embedded-larger-object-hubble_yes,clumps-embedded-larger-object-hubble_no,id_str
20,20000084,150.721991,1.625911,999.000,21.6095,20000084.jpg,31.0,17.0,3.0,14.0,...,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,/content/jpgs_euclid_complete/20000084.jpg
29,20000095,150.713586,1.626127,999.000,21.6529,20000095.jpg,33.0,9.0,5.0,1.0,...,1.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,/content/jpgs_euclid_complete/20000095.jpg
104,20000369,150.662185,1.626748,0.354,21.7621,20000369.jpg,19.0,29.0,6.0,0.0,...,1.0,7.0,5.0,6.0,1.0,5.0,8.0,11.0,2.0,/content/jpgs_euclid_complete/20000369.jpg
136,20000414,150.584715,1.626112,0.111,19.5115,20000414.jpg,33.0,8.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,/content/jpgs_euclid_complete/20000414.jpg
177,20000564,150.636166,1.627649,0.939,23.1254,20000564.jpg,30.0,6.0,6.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,/content/jpgs_euclid_complete/20000564.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84949,202172042,150.572694,1.855250,999.000,23.1246,202172042.jpg,2.0,25.0,5.0,0.0,...,0.0,0.0,0.0,4.0,5.0,1.0,0.0,1.0,0.0,/content/jpgs_euclid_complete/202172042.jpg
84950,202251062,149.997296,1.839619,999.000,22.6692,202251062.jpg,1.0,1.0,24.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,/content/jpgs_euclid_complete/202251062.jpg
84951,202735002,149.952597,2.386713,0.867,22.9417,202735002.jpg,10.0,13.0,7.0,0.0,...,1.0,0.0,3.0,7.0,0.0,3.0,9.0,4.0,8.0,/content/jpgs_euclid_complete/202735002.jpg
84952,202882042,150.085100,2.527799,1.201,23.1505,202882042.jpg,19.0,5.0,4.0,0.0,...,3.0,2.0,0.0,0.0,0.0,1.0,4.0,4.0,1.0,/content/jpgs_euclid_complete/202882042.jpg


In [ ]:
catalog_test

,survey_id,RA,DEC,Z_BEST,MAG_BEST_HI,filename,smooth-or-featured-hubble_smooth,smooth-or-featured-hubble_features,smooth-or-featured-hubble_artifact,disk-edge-on-hubble_yes,...,clump-count-hubble_2,clump-count-hubble_3,clump-count-hubble_4,clump-count-hubble_5-plus,clump-count-hubble_cant-tell,galaxy-symmetrical-hubble_yes,galaxy-symmetrical-hubble_no,clumps-embedded-larger-object-hubble_yes,clumps-embedded-larger-object-hubble_no,id_str
89,20000347,150.711232,1.627469,999.000,21.5986,20000347.jpg,29.0,10.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,/content/jpgs_euclid_complete/20000347.jpg
94,20000358,150.719030,1.627365,999.000,23.4567,20000358.jpg,29.0,8.0,13.0,1.0,...,1.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,0.0,/content/jpgs_euclid_complete/20000358.jpg
113,20000380,150.671040,1.626753,1.316,22.5915,20000380.jpg,11.0,27.0,7.0,1.0,...,3.0,10.0,5.0,5.0,2.0,3.0,21.0,15.0,9.0,/content/jpgs_euclid_complete/20000380.jpg
185,20000575,150.601022,1.627498,999.000,22.6611,20000575.jpg,36.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,/content/jpgs_euclid_complete/20000575.jpg
213,20000609,150.500419,1.626353,999.000,22.6972,20000609.jpg,23.0,7.0,7.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,3.0,3.0,0.0,/content/jpgs_euclid_complete/20000609.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84921,201827042,149.792775,2.787540,0.306,20.1270,201827042.jpg,10.0,13.0,5.0,1.0,...,3.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,1.0,/content/jpgs_euclid_complete/201827042.jpg
84927,201858002,149.558295,2.730689,999.000,23.2383,201858002.jpg,11.0,14.0,2.0,0.0,...,2.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,0.0,/content/jpgs_euclid_complete/201858002.jpg
84934,201903062,150.644545,2.807798,2.018,23.1217,201903062.jpg,6.0,11.0,16.0,0.0,...,0.0,1.0,0.0,1.0,6.0,0.0,8.0,3.0,5.0,/content/jpgs_euclid_complete/201903062.jpg
84947,201989002,149.662353,2.791116,0.232,22.1965,201989002.jpg,20.0,5.0,12.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/jpgs_euclid_complete/201989002.jpg


# Zoobot - Complete Decision Tree

### Decision Tree

In [ ]:
hubble_pairs = {
    'smooth-or-featured-hubble': ['_smooth', '_features', '_artifact'],
    'disk-edge-on-hubble': ['_yes', '_no'],
    'has-spiral-arms-hubble': ['_yes', '_no'],
    'bar-hubble': ['_yes', '_no'],
    'bulge-size-hubble': ['_none', '_just-noticeable', '_obvious', '_dominant'],
    'how-rounded-hubble': ['_completely', '_in-between', '_cigar-shaped'],
    'bulge-shape-hubble': ['_rounded', '_boxy', '_none'],
    'spiral-winding-hubble': ['_tight', '_medium', '_loose'],
    'spiral-arm-count-hubble': ['_1', '_2', '_3', '_4', '_5-plus', '_cant-tell'],
    'clumpy-appearance-hubble': ['_yes', '_no'],
    'clump-count-hubble': ['_1', '_2', '_3', '_4', '_5-plus', '_cant-tell'],
    'galaxy-symmetrical-hubble': ['_yes', '_no'],
    'clumps-embedded-larger-object-hubble': ['_yes', '_no']
}

hubble_ortho_dependencies = {
    'smooth-or-featured-hubble': None,
    'how-rounded-hubble': 'smooth-or-featured-hubble_smooth',
    'clumpy-appearance-hubble': 'smooth-or-featured-hubble_features',
    'clump-count-hubble': 'clumpy-appearance-hubble_yes',
    'galaxy-symmetrical-hubble': 'clumpy-appearance-hubble_yes',
    'clumps-embedded-larger-object-hubble': 'clumpy-appearance-hubble_yes',
    'disk-edge-on-hubble': 'clumpy-appearance-hubble_no',
    'bulge-shape-hubble': 'disk-edge-on-hubble_yes',
    'bar-hubble': 'disk-edge-on-hubble_no',
    'has-spiral-arms-hubble': 'disk-edge-on-hubble_no',
    'spiral-winding-hubble': 'disk-edge-on-hubble_no',
    'spiral-arm-count-hubble': 'disk-edge-on-hubble_no',
    'bulge-size-hubble': 'disk-edge-on-hubble_no'
}

hubble_schema = schemas.Schema(hubble_pairs,hubble_ortho_dependencies)

In [ ]:
#Full decision tree
print("Number of questions:",len(hubble_schema.questions))
print("Number of answers:",len(hubble_schema.label_cols))

Number of questions: 13
Number of answers: 40


### Specific functions

In [ ]:
def prepare_dataset_numbers(catalog,num_train,schema):
    """
        Prepares the train and validation datasets for the given schema.
    """
    train_catalog, val_catalog = train_test_split(catalog, train_size=int(0.8*num_train),test_size=int(0.2*num_train), random_state=90)

    train_image_paths = list(train_catalog['id_str'])
    val_image_paths = list(val_catalog['id_str'])

    print("Number of train galaxies:",len(train_image_paths))
    print("Number of val galaxies:",len(val_image_paths))

    train_labels = train_catalog[schema.label_cols].values
    val_labels = val_catalog[schema.label_cols].values

    train_dataset = get_image_dataset(
        train_image_paths,
        requested_img_size=initial_size,
        labels=train_labels,
        greyscale=True,
        include_id_str=False
    )

    val_dataset = get_image_dataset(
        val_image_paths,
        requested_img_size=initial_size,
        labels=val_labels,
        greyscale=True,
        include_id_str=False
    )

    transforms = default_transforms(crop_scale_bounds=(1,1),crop_ratio_bounds=(1,1),resize_after_crop=initial_size)

    train_dataset = add_transforms_to_dataset(train_dataset, transforms)
    val_dataset = add_transforms_to_dataset(val_dataset, transforms)

    train_dataset = train_dataset.shuffle(5000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return train_dataset, val_dataset

In [ ]:
def prepare_model_votes(schema):
    """
        Prepares the pretrained regression model for the finetuning.
    """
    if checkpoint_dir:
        #Pretrained model
        checkpoint_loc = os.path.join(checkpoint_dir, 'checkpoint')
        print("Loading zoobot weights from:",checkpoint_loc)
        base_model = define_model.load_model(
            input_size=initial_size,
            channels=1,
            dropout_rate=0.2,
            include_top=False,
            checkpoint_loc=checkpoint_loc
            )
    else:
        #Model without pretraining
        base_model = define_model.get_model(
            input_size=initial_size,
            output_dim=34,
            channels=1,
            dropout_rate=0.2,
            include_top=False,
            )
        print("Starting with random weights.")

    base_model.trainable = False

    new_head = tf.keras.Sequential([
        layers.InputLayer(input_shape=(1280)),
        layers.Dense(len(schema.label_cols), activation=lambda x: tf.nn.sigmoid(x) * 100. + 1.)
    ])

    model = tf.keras.Sequential([
        base_model,
        new_head
    ])

    print("Created model: Zoobot with new head")
    model.summary()

    return model

In [ ]:
def train_on_vote_labels(model, schema, train_dataset, val_dataset, save_path, lr=None, epochs=10, reduce_patience=10, patience=20):
    """
        Trains the model with the given datasets (train_dataset, val_dataset) and saves the results at the given path.
    """
    multiquestion_loss = losses.get_multiquestion_loss(schema.question_index_groups, sum_over_questions=True,\
            reduction=tf.keras.losses.Reduction.NONE)
    def loss(x, y): return multiquestion_loss(x, y) / batch_size

    extra_callbacks = []
    if lr is None:
        reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.25, patience=reduce_patience, cooldown=5, verbose=0)
        extra_callbacks = [reduce_on_plateau]
        adam_optimizer = tf.keras.optimizers.Adam()
    else:
        adam_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        extra_callbacks = []

    model.compile(loss=loss, optimizer=adam_optimizer)

    trainer = training_config.Trainer(log_dir=save_path, epochs=epochs, patience=patience)

    model = trainer.fit(model, train_dataset, val_dataset, extra_callbacks=extra_callbacks)

    return model

In [ ]:
def train_finetuning_loop(model,schema,train_dataset,val_dataset,save_path,loop_yes=True):
    print("--> Training...")
    model = train_on_vote_labels(model,schema,train_dataset,val_dataset,save_path)
    layers = ['top', 'block7', 'block6', 'block5', 'block4','block3','block2']

    if loop_yes:
        for i in range(len(layers)-1):
            print("Unfreezing model layers:",layers[:i+1])
            print("--> Training...")
            finetune.unfreeze_model(model, unfreeze_names=layers[:i+1])
            model = train_on_vote_labels(model,schema,train_dataset,val_dataset,save_path)

    print("Unfreezing all model layers.")
    print("--> Training...")
    finetune.unfreeze_model(model, unfreeze_names=[], unfreeze_all=True)
    model = train_on_vote_labels(model,schema,train_dataset,val_dataset,save_path)

    return model

In [ ]:
def make_predictions(model,pred_catalog,schema,save_path,save_name="test_predictions.csv"):
    """
        Makes predictions on images given by the 'paths' list with the given schema and saves them.
    """
    pred_image_paths = list(pred_catalog['id_str'])

    prediction_dataset = get_image_dataset(
        pred_image_paths,
        include_id_str=True,
        labels=None,
        requested_img_size=initial_size,
        greyscale=True
    )

    transforms = default_transforms(crop_scale_bounds=(1,1),crop_ratio_bounds=(1, 1),resize_after_crop=initial_size)
    prediction_dataset = add_transforms_to_dataset(prediction_dataset, transforms)

    prediction_dataset = prediction_dataset.batch(32, drop_remainder=False)

    save_loc = save_path+save_name
    predict_on_dataset.predict(prediction_dataset, model, 1, schema.label_cols, save_loc)

In [ ]:
def predict_on_dataset_reg(model,pred_catalog,schema,save_path,save_name="test_predictions.csv"):
    print("Predicting on {} galaxies...".format(len(pred_catalog)))
    make_predictions(model,pred_catalog,schema,save_path,save_name=save_name)
    print("--> Done.")
    dataset_pred = pd.read_csv(save_path+save_name)
    predictions_with_labels = pd.merge(dataset_pred, pred_catalog, on='id_str', how='inner')
    predictions_with_labels_frac = concentrations_to_vote_fractions(predictions_with_labels,schema)
    predictions_with_labels_frac.to_csv(save_path+save_name,index=False)
    print("All results saved at {}".format(save_path+save_name))

In [ ]:
def concentrations_to_vote_fractions(dataset_pred,schema,with_labels=True):
    """
        Transforms the given concentrations into vote fractions for the given
        binary schema and save them.
    """
    answers = list(schema.label_cols)
    answers_pred = []
    for i in range(len(answers)): answers_pred.append(answers[i] + "_pred")
    if with_labels:
        answer_list = [answers,answers_pred]
    else:
        answer_list = [answers_pred]
    answer_indices = list(schema.question_index_groups)
    print("Calculating vote fractions for {} questions with {} answers in total...".format(len(answer_indices),len(answers)))

    print("--> For {} galaxies".format(len(dataset_pred)))
    for n in range(len(answer_list)):
        for i in range(len(answer_indices)):
            #The concentrations for all answers (index 0 is answer 0, ...)
            data_concentrations = []
            for j in range(answer_indices[i][0],answer_indices[i][1]+1):
                data_votes = list(dataset_pred[answer_list[n][j]])
                data_conc = []
                for k in range(len(data_votes)):
                    data_conc.append(data_votes[k])
                data_concentrations.append(data_conc)
            data_concentrations = np.array(data_concentrations)
            #Calculate fractions
            total_votes = data_concentrations.sum(axis=0)
            data_frac = data_concentrations/total_votes
            for j in range(answer_indices[i][0],answer_indices[i][1]+1):
                dataset_pred[answer_list[n][j]+"_frac"] = data_frac[j-answer_indices[i][0],:]
                question = answer_list[n][j].split("_")[0]
            if n==0: dataset_pred[question+"_number"] = total_votes

    return dataset_pred

In [ ]:
def mean_deviation_half(schema,save_path,save_name,model_name="Zoobot - Euclid"):
    data = pd.read_csv(save_path+save_name)

    questions = list(schema.question_answer_pairs.keys())
    answers = schema.label_cols
    answer_indices = schema.question_index_groups

    print("Calculating deviations...")

    #Get number of volunteers
    number_votes = data["smooth-or-featured-hubble_number"]
    half_num_votes = np.array(number_votes)/2

    #Calculate deviations
    deviations_answers, num_answers = [], []
    labels_complete, predictions_complete = [], []
    for i in range(len(answers)):
        predictions, labels = [], []
        question = answers[i].split("_")[0]
        for j in range(len(data)):
            if data[question+"_number"].iloc[j] >= half_num_votes[j]:
                predictions.append(data[answers[i]+"_pred_frac"].iloc[j])
                labels.append(data[answers[i]+"_frac"].iloc[j])
        deviations = np.abs(np.array(predictions)-np.array(labels))
        num_answers.append(len(deviations))
        deviations_answers.append(np.mean(deviations))
        labels_complete.append(labels)
        predictions_complete.append(predictions)

    #Plot
    plt.figure(figsize=(8,12))
    for i in range(len(answer_indices)):
        plt.barh(answers[answer_indices[i][0]:answer_indices[i][1]+1],deviations_answers[answer_indices[i][0]:answer_indices[i][1]+1])
        plt.text(deviations_answers[answer_indices[i][0]:answer_indices[i][1]+1][0] + 0.002,\
                    answer_indices[i][0] + .2,str(num_answers[answer_indices[i][0]:answer_indices[i][1]+1][0]),fontsize="small",color="gray")
    plt.gca().invert_yaxis()
    plt.margins(0.01)
    plt.xlabel("Vote Fraction Mean Deviation")
    plt.grid(axis="x")
    plt.title(model_name)
    plt.vlines(0.12,0,40,color="black",linestyle="--",linewidth=1)
    #plt.savefig(save_path+"deviations_half.png",bbox_inches='tight')
    plt.show()

    print("Average deviation:",np.mean(deviations_answers))

    return predictions_complete,labels_complete,deviations_answers,num_answers

In [ ]:
def apply_model_regression(catalog_train,catalog_test,schema,folder,weights,num_galaxies):
    """
        Finetunes the pretrained Zoobot model to a given problem and evaluates the performance.
        Predicts number of volunteers choosing certain answer for all questions simultaneously.
    """
    print("--------- ZOOBOT --------")
    print("____________________________________\n")
    save_path = "/content/drive/MyDrive/MPE/ZoobotEuclid/Results/euclid/{}/weights_{}/num_galaxies_{}/".format(folder,weights,str(num_galaxies))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    print("APPLY MODEL TO QUESTIONS: {} ".format(list(schema.question_answer_pairs.keys())))
    print("Saving all results to:",save_path)
    print("____________________________________\n")
    print("---- 1) CREATE DATASETS ----")
    print("____________________________________\n")
    train_dataset, val_dataset = prepare_dataset_numbers(catalog_train,num_galaxies,schema)
    print("____________________________________\n")
    print("---- 2) PREPARE MODEL ----")
    print("____________________________________\n")
    model = prepare_model_votes(schema)
    print("____________________________________\n")
    print("---- 3) TRAIN MODEL ----")
    print("____________________________________\n")
    trained_model = train_finetuning_loop(model,schema,train_dataset,val_dataset,save_path,loop_yes=False)
    print("____________________________________\n")
    print("---- 4) MAKE PREDICTIONS ON DATASETS ----")
    print("____________________________________\n")
    predict_on_dataset_reg(trained_model,catalog_test,schema,save_path,save_name="test_predictions.csv")
    mean_deviation_half(schema,save_path,save_name="test_predictions.csv",model_name="Zoobot - Euclid")

### Apply to the questions

In [ ]:
#Number of galaxies
num_galaxies = [len(catalog_train_val)]
for i in range(len(num_galaxies)):
    apply_model_regression(catalog_train_val,catalog_test,hubble_schema,"complete","D",num_galaxies[i])